<a href="https://colab.research.google.com/github/JGranda11/Proyecto-ai4eng20241/blob/main/07%20-%20preprocesado%20forma%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Preprocesamiento
En el siguiente cuaderno se quiere mostrar que realizacione sy consideraciones se tuvieron en cuenta para aplicar las operaciones de limpieza y preprocesado de datos.

## Cargar el Dataset

In [ ]:
#Cargar el token de Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jhonatangranda","key":"7dd2bc4d9b9fc9fcaeb5d991e5472b44"}'}

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download udea-ai4eng-20241

 94% 20.0M/21.2M [00:01<00:00, 30.9MB/s]
100% 21.2M/21.2M [00:01<00:00, 19.9MB/s]


In [ ]:
#Descomprimiendo los archivos
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# Cargar dataset

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


#Consideraciones
Del proceso de exploración de los datos, se infiere que el dataset en su composición tiene:

* Columnas repetidas:
__FAMI_TIENEINTERNET__ y __FAMI_TIENEINTERNET.1__

Adeás necesito categorizar varias columnas que representan un orden en su estructura
* ESTU_VALORMATRICULAUNIVERSIDAD
* ESTU_HORASSEMANATRABAJA
* FAMI_ESTRATOVIVIENDA
* FAMI_EDUCACIONPADRE
* FAMI_EDUCACIONMADRE

 Por lo que asignar numericamente un numero consecutivo, representa una categoria

In [ ]:
#Se hace la asignación, para el cambio de nombre de la columna repetida
train = train.rename(columns={'FAMI_TIENEINTERNET.1': 'FAMI_TIENEINTERNET2'})

In [ ]:
train.FAMI_TIENEINTERNET.value_counts()
#Columna original

FAMI_TIENEINTERNET
Si    592514
No     73357
Name: count, dtype: int64

In [ ]:
train.FAMI_TIENEINTERNET2.value_counts()
#COLUMNA DUPLICADA

FAMI_TIENEINTERNET2
Si    592514
No     73357
Name: count, dtype: int64

In [ ]:
validar = train['FAMI_TIENEINTERNET'].equals(train['FAMI_TIENEINTERNET2'])
print(validar)

True


In [ ]:
train = train.drop('FAMI_TIENEINTERNET2', axis=1)

In [ ]:
train.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto
1,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo
2,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo
3,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto
4,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_TIENELAVADORA              652727 non-null  object
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object
 12  ESTU_PAGOMATRICULAPROPIO      

Evaluación general de todos los datos nulos

In [ ]:
k = train.isna().sum()
k[k!=0]


ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
dtype: int64

In [ ]:
#Borramos ID desde el principio
train = train.drop('ID', axis=1)

## Manejo de la columna PERIODO
Esta columna presenta 9 distintas posibilidades de ocurrencia, se quiere corregir y agrupar de mejor manera por año.

In [ ]:
train['PERIODO'].unique()

array([20212, 20203, 20195, 20183, 20194, 20213, 20184, 20202, 20196])

In [ ]:
train.PERIODO.value_counts()

PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

In [ ]:
train = train.drop('PERIODO', axis=1)

## Manejo de la columna ESTU_PRGM_ACADEMICO

In [ ]:
#La columna ESTU_PRGM_ACADEMICO es la que representa un mayor numero de variedad en el rango de sus posibilidades
train["ESTU_PRGM_ACADEMICO"].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA                           1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES                    1
ADMINISTRACION DE COMERCIO EXTERIOR                           1
LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES        1
PROFESIONAL EN INVESTIGACIÓN CRIMINAL                         1
Name: count, Length: 948, dtype: int64

In [ ]:
train['ESTU_PRGM_ACADEMICO'].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA                           1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES                    1
ADMINISTRACION DE COMERCIO EXTERIOR                           1
LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES        1
PROFESIONAL EN INVESTIGACIÓN CRIMINAL                         1
Name: count, Length: 948, dtype: int64

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.2 MB/s eta 0:00:00


Se hacen ciertas consideraciones para eliminar varios acentos, y caracteres es las posibles respuestas de la columna

In [ ]:
from unidecode import unidecode
import re

train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].str.upper() # Todo mayúsculas
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(x).strip()) # Eliminar tildes
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ',x).strip()) # Eliminar dobles espacios
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'^PROFESIONAL EN\s+', '', x).strip()) # "PROFESIONAL EN ..." redundante
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGENIER\?A', 'INGENIERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGENIIERIA', 'INGENIERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'EDUCACI\?N\b', 'EDUCACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'RECREACI\?N\b', 'RECREACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'F\?SICA\b', 'FISICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ESC\?NICAS\b', 'ESCENICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MATEM\?TICAS\b', 'MATEMATICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOF\?A\b', 'FILOSOFIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?O\b', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?\?O\b', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?N\b', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?\?N\b', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GR\?\?FICA\b', 'GRAFICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'EL\?CTRICA\b', 'ELECTRICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'QU\?MICA\b', 'QUIMICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FARMAC\?UTICA\b', 'FARMACEUTICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACI\?N\b', 'ADMINISTRACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'P\?BLICA\b', 'PUBLICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGL\?S\b', 'INGLES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'\?NFASIS\b', 'ENFASIS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GESTI\?N\b', 'GESTION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'M\?SICA\b', 'MUSICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ECONOM\?A\b', 'ECONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GASTRONOM\?A\b', 'GASTRONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'B\?SICA\b', 'BASICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GEOLOG\?A\b', 'GEOLOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'PEDAGOG\?A\b', 'PEDAGOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ART\?STICA\b', 'ARTISTICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LOG\?STICA\b', 'LOGISTICA', regex=True)
# Reemplazar cualquier coincidencia de "{NOMBRE + AGG} ..." solo por "{NOMBRE} mas general"
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*INGENIERIA.*', 'INGENIERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISEÑO.*', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISENO.*', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LICENCIATURA.*', 'LICENCIATURA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CIENCIA.*', 'CIENCIAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACION.*', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GESTION.*', 'GESTION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOFIA.*', 'FILOSOFIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'QUIMICA.*', 'QUIMICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'NEGOCIOS.*', 'NEGOCIOS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*MERCAD.*', 'MERCADEO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MATEMATICAS.*', 'MATEMATICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*ARTE.*', 'ARTES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ESTUDIOS.*', 'ESTUDIOS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FINANZAS.*', 'FINANZAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DIRECCION.*', 'DIRECCION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'RELACIONES.*', 'RELACIONES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GEO.*', 'GEOGRAFIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'PROFESIONAL.*', 'PROFESIONAL', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LENG.*', 'LENGUAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ADMIN.*', 'ADMINISTRACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*DEPORT.*', 'DEPORTE', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*BIOLOGIA.*', 'BIOLOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CINE.*', 'CINE', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CONTADURIA.*', 'CONTADURIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMERCIO.*', 'COMERCIO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*LITERA.*', 'LITERATURA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*FILOL.*', 'FILOLOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'PUBLICI.*', 'PUBLICIDAD', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MARK.*', 'MERCADEO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'HISTORIA.*', 'HISTORIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*MUSICA.*', 'MUSICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*VETE.*', 'VETERINARIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*ZOO.*', 'VETERINARIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*GASTR.*', 'GASTRONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*QUIRURGICA.*', 'MEDICINA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ECONO.*', 'ECONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GEREN.*', 'GERENCIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'HOTELERIA.*', 'HOTELERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*PSICOLO.*', 'PSICOLOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'.*AGRO.*', 'AGRONOMIA', regex=True)


In [ ]:
train['ESTU_PRGM_ACADEMICO'].isnull().any()
#Se verifica que la columna no tenga valores nulos

False

Lo que se hizo fue reducir el rango de las columna 'ESTU_PRGM_ACADEMICO', para que se generan muchas columnas al aplicar Codificación, mayormente llenas de ceros

In [ ]:
len(train['ESTU_PRGM_ACADEMICO'].unique())

110

In [ ]:
carreras_agrupadas = ['INGENIERIA','ADMINISTRACION','DISEÑO','LICENCIATURA','CIENCIAS','COMUNICACION','GESTION','FILOSOFIA','QUIMICA','NEGOCIOS','MERCADEO',
                       'MATEMATICAS','ARTES','ESTUDIOS','FINANZAS','CONSTRUCCION','DIRECCION','RELACIONES','GEOGRAFIA','PROFESIONAL','LENGUAS', 'DEPORTE'
                       'BIOLOGIA','CINE','CONTADURIA','COMERCIO','LITERATURA','FILOLOGIA','PUBLICIDAD','HISTORIA','MUSICA','VETERINARIA','GATRONOMIA'
                       'MEDICINA', 'ECONOMIA','GERENCIA','HOTELERIA','PSICOLOGIA','AGRONOMIA']
len(carreras_agrupadas)

37

In [ ]:
carreras_sobrantes = [elem for elem in train['ESTU_PRGM_ACADEMICO'] if elem not in carreras_agrupadas]

condicion = train['ESTU_PRGM_ACADEMICO'].isin(carreras_sobrantes)
train.loc[condicion, 'ESTU_PRGM_ACADEMICO'] = 'Otras'




In [ ]:
academico=train['ESTU_PRGM_ACADEMICO'].unique()
len(academico)
#academico

36

In [ ]:
##CODIFICAR PRGM ACADEMICO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(train[['ESTU_PRGM_ACADEMICO']])
#generar codificacion

arregloPr = codificadorPr.toarray()
programa = pd.DataFrame(arregloPr,
                       columns = encode.categories_)

train = pd.concat([train, programa], axis = 'columns')
train = train.drop('ESTU_PRGM_ACADEMICO', axis=1)

In [ ]:
train.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,"(MERCADEO,)","(MUSICA,)","(NEGOCIOS,)","(Otras,)","(PROFESIONAL,)","(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)"
0,Otras,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Otras,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MERCADEO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADMINISTRACION,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
ccols = [i for i in train.columns if not i in train._get_numeric_data()]
print (ccols)

['ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']


In [ ]:
for c in ccols:
    print ("%10s"%c, np.unique(train[c].dropna()))

ESTU_PRGM_DEPARTAMENTO ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOGOTÁ' 'BOLIVAR' 'BOYACA'
 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA'
 'CUNDINAMARCA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META'
 'NARIÑO' 'NORTE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA' 'SAN ANDRES'
 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE' 'VAUPES']
ESTU_VALORMATRICULAUNIVERSIDAD ['Entre 1 millón y menos de 2.5 millones'
 'Entre 2.5 millones y menos de 4 millones'
 'Entre 4 millones y menos de 5.5 millones'
 'Entre 5.5 millones y menos de 7 millones'
 'Entre 500 mil y menos de 1 millón' 'Menos de 500 mil'
 'Más de 7 millones' 'No pagó matrícula']
ESTU_HORASSEMANATRABAJA ['0' 'Entre 11 y 20 horas' 'Entre 21 y 30 horas' 'Menos de 10 horas'
 'Más de 30 horas']
FAMI_ESTRATOVIVIENDA ['Estrato 1' 'Estrato 2' 'Estrato 3' 'Estrato 4' 'Estrato 5' 'Estrato 6'
 'Sin Estrato']
FAMI_TIENEINTERNET ['No' 'Si']
FAMI_EDUCACIONPADRE ['Educación profesional completa' 'Educación profesional incompleta'
 'Ni

## Manejo de la columna ESTU_PRGM_DEPARTAMENTO
Con el manejo de esta columna lo que se quiere es agrupar por regiones dichos departamentos, es decir, pasar de tener 32 tipos de departamentos, a solo cinco regiones que los agrupen

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['BOGOTÁ', 'ATLANTICO', 'SANTANDER', 'ANTIOQUIA', 'HUILA', 'SUCRE',
       'CAQUETA', 'CUNDINAMARCA', 'BOLIVAR', 'TOLIMA', 'VALLE', 'QUINDIO',
       'RISARALDA', 'CORDOBA', 'META', 'LA GUAJIRA', 'BOYACA', 'NARIÑO',
       'CAUCA', 'NORTE SANTANDER', 'CESAR', 'PUTUMAYO', 'CALDAS',
       'MAGDALENA', 'CHOCO', 'CASANARE', 'ARAUCA', 'GUAVIARE', 'AMAZONAS',
       'VAUPES', 'SAN ANDRES'], dtype=object)

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ANTIOQUIA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOYACA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOGOTÁ', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CALDAS', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CUNDINAMARCA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'HUILA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NORTE SANTANDER', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'QUINDIO', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'RISARALDA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SANTANDER', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'TOLIMA', 'ANDINA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'AMAZONAS', 'AMAZONA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAQUETA', 'AMAZONA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VAUPES', 'AMAZONA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'GUAVIARE', 'AMAZONA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'PUTUMAYO', 'AMAZONA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VALLE', 'PACIFICA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CHOCO', 'PACIFICA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAUCA', 'PACIFICA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NARIÑO', 'PACIFICA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ATLANTICO', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOLIVAR', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CESAR', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CORDOBA', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'LA GUAJIRA', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'MAGDALENA', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SUCRE', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SAN ANDRES', 'CARIBE')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ARAUCA', 'ORINOQUIA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CASANARE', 'ORINOQUIA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'META', 'ORINOQUIA')
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VICHADA', 'ORINOQUIA')

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['ANDINA', 'CARIBE', 'AMAZONA', 'PACIFICA', 'ORINOQUIA'],
      dtype=object)

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].isna().sum()

0

In [ ]:
##CODIFICAR PRGM DEPARTAMENTO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorDepto = encode.fit_transform(train[['ESTU_PRGM_DEPARTAMENTO']])
#generar codificacion

arregloDepto = codificadorDepto.toarray()
departamento = pd.DataFrame(arregloDepto,
                       columns = encode.categories_)

train = pd.concat([train, departamento], axis = 'columns')
train = train.drop('ESTU_PRGM_DEPARTAMENTO', axis=1)

In [ ]:
train.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Manejo de la columna ESTU_VALORMATRICULAUNIVERSIDAD
La columna presenta una posibilidad de 8 valores a la hroa de elegir cualquier registro y 6287 datos son nulos

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()


array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

6287

Tenemos una columna que presenta 8 posibles categorias, y que la categoria con mayor frecuencia, es un valor de matricula entre 1 y 2.5 M.

Se decide rellenar los datos faltantes, asumiendo que aquellos estudiantes que no tengan un valor asignado, es como si no realizaran un pago


In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')

In [ ]:
#Realizamos el Encode para este columna
valores_matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']

enc = OrdinalEncoder(categories = [valores_matricula])

train['ESTU_VALORMATRICULAUNIVERSIDAD'] = enc.fit_transform(train[['ESTU_VALORMATRICULAUNIVERSIDAD']])

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
3.0    204048
4.0    127430
1.0     80263
2.0     78704
5.0     69736
7.0     68014
6.0     38490
0.0     25815
Name: count, dtype: int64

## Manejo de la columna ESTU_HORASSEMANATRABAJA
La columna presenta 5 categorias, y presenta 30857 datos nulos, por lo que no debo realizar alguna reparacion, paar este caso asumimos que si el campo esta vacio, es por que el estudiante no tiene unas horas de trabajo


In [ ]:
train['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [ ]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64

In [ ]:
train['ESTU_HORASSEMANATRABAJA'].isna().sum()

30857

In [ ]:
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [ ]:
horas_trabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']

enc = OrdinalEncoder(categories = [horas_trabajo])

train['ESTU_HORASSEMANATRABAJA'] = enc.fit_transform(train[['ESTU_HORASSEMANATRABAJA']])

In [ ]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
4.0    249352
0.0    147407
2.0    115857
3.0     92693
1.0     87191
Name: count, dtype: int64

## Manejo de la columna FAMI_ESTRATOVIVIENDA
Dicha columna presenta 7 categorias, y ademas de eso un total de 32137 datos nulos, la estrategia que se plantea para repaar dichos datos nulos se baso en ver cual era la categoria con mayor frecuencia

In [ ]:
train['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [ ]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
Estrato 2      232671
Estrato 3      210685
Estrato 1      111991
Estrato 4       65514
Estrato 5       23608
Estrato 6       12605
Sin Estrato      3289
Name: count, dtype: int64

In [ ]:
train['FAMI_ESTRATOVIVIENDA'].isna().sum()

32137

In [ ]:
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [ ]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']

enc = OrdinalEncoder(categories = [estratos])

train['FAMI_ESTRATOVIVIENDA'] = enc.fit_transform(train[['FAMI_ESTRATOVIVIENDA']])

In [ ]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
2.0    264808
3.0    210685
1.0    111991
4.0     65514
5.0     23608
6.0     12605
0.0      3289
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEINTERNET
Para este caso, tengo solo dos posibles clases, por lo que debo codificar de manera binaria, ademas tengo 26629 datos nuelos, y se decide clasificar dichos datos que no aparecen, como un 'NO'

In [ ]:
train['FAMI_TIENEINTERNET'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
train['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
Si    592514
No     73357
Name: count, dtype: int64

In [ ]:
train['FAMI_TIENEINTERNET'].isna().sum()

26629

In [ ]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].fillna("No")

In [ ]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'Si', 1.)
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'No', 0.)

In [ ]:
train['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
1.0    592514
0.0     99986
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENELAVADORA
Esta columna tambien presenta una posibilidad de valores de 2 rangos, por lo que dicha encodificación va a ser binaria, ademas contienen 39773 registros nulos

In [ ]:
train['FAMI_TIENELAVADORA'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
train['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
Si    563390
No     89337
Name: count, dtype: int64

In [ ]:
train['FAMI_TIENELAVADORA'].isna().sum()

39773

In [ ]:
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].fillna("No")

In [ ]:
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].replace(r'Si', 1.)
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].replace(r'No', 0.)

In [ ]:
train['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
1.0    563390
0.0    129110
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEAUTOMOVIL
La columna presenta una distinción entre valores de tipo binario, por lo que todos los datos pueden tomar solo dos valores, y presenta 43623 registros nulos

In [ ]:
train['FAMI_TIENEAUTOMOVIL'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
train['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
No    412606
Si    236271
Name: count, dtype: int64

In [ ]:
train['FAMI_TIENEAUTOMOVIL'].isna().sum()

43623

In [ ]:
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].fillna("No")

In [ ]:
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].replace(r'Si', 1.)
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].replace(r'No', 0.)

In [ ]:
train['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
0.0    456229
1.0    236271
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENECOMPUTADOR
Comportmaiento binario.
Presenta 38103 registros nulos

In [ ]:
train['FAMI_TIENECOMPUTADOR'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
train['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
Si    597670
No     56727
Name: count, dtype: int64

In [ ]:
train['FAMI_TIENECOMPUTADOR'].isna().sum()

38103

In [ ]:
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].fillna("No")

In [ ]:
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].replace(r'Si', 1.)
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].replace(r'No', 0.)

In [ ]:
train['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
1.0    597670
0.0     94830
Name: count, dtype: int64

## Manejo de la columna ESTU_PAGOMATRICULAPROPIO
Esta columna presenta un rango binario en su conjunto de posibles valores, y un total de 6498 datos nulos.
Por el momento la decisión para hcaer la reparación, se elige de asignar 'NO' a los datos nulos

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'].unique()

array(['No', 'Si', nan], dtype=object)

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
No    382201
Si    303801
Name: count, dtype: int64

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

6498

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna("No")

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.)
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.)

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
0.0    388699
1.0    303801
Name: count, dtype: int64

## Manejo de la columna ESTU_PRIVADO_LIBERTAD
Esta columna tiene un comportamiento binario, y no presenta datos nulos, se tiene pendiente el proceso de evaluar la presencia de esta variable en el modelo, ya que el 99% de los datos son registros con negación de la condición de la columna

In [ ]:
train['ESTU_PRIVADO_LIBERTAD'].unique()

array(['N', 'S'], dtype=object)

In [ ]:
train['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
N    692466
S        34
Name: count, dtype: int64

In [ ]:
train = train.drop('ESTU_PRIVADO_LIBERTAD', axis=1)

## Manejo de la COLUMNA FAMI_EDUCACIONPADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, junto con lso datos nulos, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [ ]:
train['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [ ]:
train['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado                                44169
Educación profesional incompleta         27084
Técnica o tecnológica incompleta         22552
Ninguno                                  22008
No sabe                                  16592
No Aplica                                 9229
Name: count, dtype: int64

* 0 No aplica
* 1 primaria incompleta
* 2 primaria completa
* 3 Secundaria (Bachillerato) incompleta  
* 4 Secundaria (Bachillerato) completa
* 5 Tecnica o tecnolgia incompleta
* 6 Tecnica o tecnolgia incompleta
* 7 Educacion profesional incompleta
* 8 Educacion profesional completa
* 9 postgrado


In [ ]:
train['FAMI_EDUCACIONPADRE'].isna().sum()

23178

In [ ]:
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].replace(r'Ninguno', 'No Aplica')
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].replace(r'No sabe', 'No Aplica')

In [ ]:
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [ ]:
train['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
No Aplica                                71007
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado                                44169
Educación profesional incompleta         27084
Técnica o tecnológica incompleta         22552
Name: count, dtype: int64

Ahora toca codificar la columna

In [ ]:

nivelesPadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']

enc = OrdinalEncoder(categories = [nivelesPadre])

train['FAMI_EDUCACIONPADRE'] = enc.fit_transform(train[['FAMI_EDUCACIONPADRE']])

In [ ]:
train['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
4.0    128289
1.0    125675
8.0     83117
3.0     71654
0.0     71007
6.0     62995
2.0     55958
9.0     44169
7.0     27084
5.0     22552
Name: count, dtype: int64

In [ ]:
train.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,6.0,1.0,3.0,1.0,5.0,1.0,1.0,0.0,1.0,Postgrado,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,4.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,1.0,Técnica o tecnológica incompleta,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.0,4.0,3.0,1.0,4.0,1.0,0.0,0.0,0.0,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,5.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.0,3.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,Primaria completa,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Manejo de la columna FAMI_EDUCACIONMADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [ ]:
train['FAMI_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No Aplica', 'No sabe'], dtype=object)

In [ ]:
train['FAMI_EDUCACIONMADRE'].value_counts()

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141744
Primaria incompleta                      99420
Técnica o tecnológica completa           89542
Educación profesional completa           85326
Secundaria (Bachillerato) incompleta     81012
Primaria completa                        56125
Postgrado                                46246
Técnica o tecnológica incompleta         27533
Educación profesional incompleta         22470
Ninguno                                  14483
No sabe                                   3017
No Aplica                                 1918
Name: count, dtype: int64

In [ ]:
train['FAMI_EDUCACIONMADRE'].isna().sum()

23664

In [ ]:
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].replace(r'Ninguno', 'No Aplica')
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].replace(r'No sabe', 'No Aplica')

In [ ]:
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [ ]:
nivelesMadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']

enc = OrdinalEncoder(categories = [nivelesMadre])

train['FAMI_EDUCACIONMADRE'] = enc.fit_transform(train[['FAMI_EDUCACIONMADRE']])

In [ ]:
train['FAMI_EDUCACIONMADRE'].value_counts()

FAMI_EDUCACIONMADRE
4.0    141744
1.0     99420
6.0     89542
8.0     85326
3.0     81012
2.0     56125
9.0     46246
0.0     43082
5.0     27533
7.0     22470
Name: count, dtype: int64

In [ ]:
train.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,6.0,1.0,3.0,1.0,5.0,1.0,1.0,0.0,1.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,4.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.0,4.0,3.0,1.0,4.0,1.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,5.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.0,3.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
train.shape

(692500, 52)

In [ ]:
np.unique(train['RENDIMIENTO_GLOBAL'].values)

array(['alto', 'bajo', 'medio-alto', 'medio-bajo'], dtype=object)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
rendimientos = ['bajo', 'medio-bajo', 'medio-alto', 'alto']
cod = OrdinalEncoder(categories = [rendimientos])

train['RENDIMIENTO_GLOBAL'] = cod.fit_transform(train[['RENDIMIENTO_GLOBAL']])

In [ ]:
#Dataset con 58 columnas, y los mismos 692500 registros

In [ ]:
print(len(train.columns))
train.columns

52


Index(['ESTU_VALORMATRICULAUNIVERSIDAD',        'ESTU_HORASSEMANATRABAJA',
                 'FAMI_ESTRATOVIVIENDA',             'FAMI_TIENEINTERNET',
                  'FAMI_EDUCACIONPADRE',             'FAMI_TIENELAVADORA',
                  'FAMI_TIENEAUTOMOVIL',       'ESTU_PAGOMATRICULAPROPIO',
                 'FAMI_TIENECOMPUTADOR',            'FAMI_EDUCACIONMADRE',
                   'RENDIMIENTO_GLOBAL',              ('ADMINISTRACION',),
                         ('AGRONOMIA',),                       ('ARTES',),
                          ('CIENCIAS',),                        ('CINE',),
                          ('COMERCIO',),                ('COMUNICACION',),
                      ('CONSTRUCCION',),                  ('CONTADURIA',),
                         ('DIRECCION',),                      ('DISEÑO',),
                          ('ECONOMIA',),                    ('ESTUDIOS',),
                         ('FILOLOGIA',),                   ('FILOSOFIA',),
                         

In [ ]:
archivo = train.to_csv("trainP.csv", index = False)

In [ ]:
!head trainP.csv

ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,"('ADMINISTRACION',)","('AGRONOMIA',)","('ARTES',)","('CIENCIAS',)","('CINE',)","('COMERCIO',)","('COMUNICACION',)","('CONSTRUCCION',)","('CONTADURIA',)","('DIRECCION',)","('DISEÑO',)","('ECONOMIA',)","('ESTUDIOS',)","('FILOLOGIA',)","('FILOSOFIA',)","('FINANZAS',)","('GEOGRAFIA',)","('GERENCIA',)","('GESTION',)","('HISTORIA',)","('HOTELERIA',)","('INGENIERIA',)","('LENGUAS',)","('LICENCIATURA',)","('LITERATURA',)","('MATEMATICAS',)","('MERCADEO',)","('MUSICA',)","('NEGOCIOS',)","('Otras',)","('PROFESIONAL',)","('PSICOLOGIA',)","('PUBLICIDAD',)","('QUIMICA',)","('RELACIONES',)","('VETERINARIA',)","('AMAZONA',)","('ANDINA',)","('CARIBE',)","('ORINOQUIA',)","('PACIFICA',)"
6.0,1.0,3.0,1.0,5.0,1.0,1.0,0.0,1.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, ShuffleSplit, KFold, cross_val_score, train_test_split

In [ ]:
X = train.drop(['RENDIMIENTO_GLOBAL'], axis=1).values

In [ ]:
pd.options.display.max_columns = 0
X

array([[6., 1., 3., ..., 0., 0., 0.],
       [4., 0., 3., ..., 1., 0., 0.],
       [4., 4., 3., ..., 0., 0., 0.],
       ...,
       [3., 1., 3., ..., 0., 0., 0.],
       [4., 1., 1., ..., 0., 0., 0.],
       [6., 4., 4., ..., 0., 0., 0.]])

In [ ]:
np.unique(y)

array([0., 1., 2., 3.])

In [ ]:
np.unique(X)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
y = train['RENDIMIENTO_GLOBAL'].values

(692500, 51) (692500,)
test size 0.30
val size is 0.43 (relative to 0.70) 
(484750, 51) (207750, 51)


In [ ]:
estimator1 = RandomForestClassifier()
z1 = cross_validate(estimator1, Xtr, ytr, return_train_score=True, return_estimator=True,
                    scoring='accuracy', cv=ShuffleSplit(n_splits=10, test_size=0.3))

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
z1

In [ ]:
estimator1.fit(Xtr, ytr)
estimator1.score(Xtr, ytr), estimator1.score(Xts, yts)